# 交差検証を楽に書こうぜ
- `cross_validate()`
- `GridSearchCV`
- `Pipeline`


In [20]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

# データの準備

In [21]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

X = cancer.data
y = cancer.target

print(X.shape)

(569, 30)


In [22]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X,
                                                          y,
                                                          train_size=0.80,
                                                          random_state=42)

In [23]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42)

# `cross_validate()`


In [24]:
from sklearn.model_selection import cross_validate

In [25]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


cross_validate(estimator=SVC(),
               X=X_trainval,
               y=y_trainval,
               scoring=['accuracy', 'recall'],
               cv=skf,
               n_jobs=-1)

{'fit_time': array([0.00715017, 0.00724006, 0.00734115, 0.00881195, 0.00721788]),
 'score_time': array([0.00319004, 0.00322199, 0.00307679, 0.00387311, 0.00308919]),
 'test_accuracy': array([0.63043478, 0.62637363, 0.62637363, 0.62637363, 0.63333333]),
 'test_recall': array([1., 1., 1., 1., 1.])}

# `GridSearchCV()`

In [26]:
from sklearn.model_selection import GridSearchCV

svm_param_grid = {
    'C': [10**i for i in range(-4, 4)],
    'gamma': [10**i for i in range(-4, 4)],
}

grid_svm = GridSearchCV(estimator=SVC(),
                        param_grid=svm_param_grid,
                        cv=skf,
                        scoring='accuracy',
                        n_jobs=-1)

In [27]:
# 学習
grid_svm.fit(X_trainval, y_trainval)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [28]:
grid_svm.best_score_, grid_svm.best_params_

(0.9494505494505494, {'C': 10, 'gamma': 0.0001})

In [29]:
grid_svm.cv_results_

{'mean_fit_time': array([0.00674171, 0.0066134 , 0.00622153, 0.00579834, 0.00527525,
        0.00485592, 0.00382438, 0.00373287, 0.00539799, 0.00608454,
        0.00535245, 0.00433106, 0.00585146, 0.00565033, 0.0064858 ,
        0.00577469, 0.00598464, 0.00675135, 0.00848942, 0.00756316,
        0.00682149, 0.00665097, 0.00765338, 0.00951705, 0.00599008,
        0.00715837, 0.00864906, 0.00818882, 0.00791125, 0.00745807,
        0.00744963, 0.00731711, 0.00428023, 0.00844598, 0.00915217,
        0.00876393, 0.00760016, 0.00765858, 0.00799799, 0.00874677,
        0.00415454, 0.00931377, 0.00998058, 0.00937777, 0.00851774,
        0.0083353 , 0.00835528, 0.00833893, 0.00510869, 0.00837307,
        0.00987639, 0.0093636 , 0.00851483, 0.00838518, 0.00869575,
        0.00841484, 0.00566816, 0.00817747, 0.01019387, 0.00925913,
        0.00848584, 0.00846448, 0.00833721, 0.00711937]),
 'std_fit_time': array([9.05553060e-05, 2.39510734e-04, 6.21836992e-04, 1.37907628e-03,
        1.25239014e-0

In [30]:
grid_svm.score(X_test, y_test)

0.9473684210526315

# `Pipeline()`

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# Pipelineで使った Key に __ をつけるぞ〜
svm_param_pipe = {
    'svm__C': [10**i for i in range(-4, 4)],
    'svm__gamma': [10**i for i in range(-4, 4)],
}

grid_svm_pipe = GridSearchCV(pipe,
                             param_grid=svm_param_pipe,
                             cv=skf,
                             scoring='accuracy',
                             n_jobs=-1)

grid_svm_pipe.fit(X_trainval, y_trainval)

print(grid_svm_pipe.score(X_test, y_test))

0.9824561403508771
